##  Talent Scheduling 

1. Welche Entscheidungen / Handlungen / Aktionen sind möglich?
- eine Beschreibung des aktuellen **Zustands**:
  - TSP: die bisherige Tour
  - Rucksackproblem: das bisher eingepackte Gewicht
  - *Talent Scheduling*: die Reihenfolge der zu drehenden Szenen 
- die "Regeln" / Restriktionen, die beschreiben, **welche Handlungen in einem Zustand zulässig sind**
  - TSP: kein erneuter Besuch einer bereits besuchten Stadt
  - Rucksackproblem: Gegenstand einpacken geht nur, wenn das Gesamtgewicht nicht überschritten wird
  - *Talent Scheduling*: eine Szene wird nur einmal gedreht
 
2. Welche der möglichen Entscheidungen sollte gewählt werden?
- z.B. der Wert der neuen Teillösung:
  - TSP: Distanz der (Teil-)Tour nach Hinzunahme des Nacharn
  - Rucksackproblem: Gesamtwert der Lösung nach Hinzunahme eines Elements
  - *Talent Scheduling*: Die Szenen, die am meisten Kosten, werden zuerst gedreht(= die Szenen absteigend nach Kosten sortieren)

In [1]:
#IMPORTs
import numpy as np
import matplotlib.pyplot as plt

#numba
from numba import njit, jit
from numba.typed import List

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [2]:
class ts_udp:
    """
    UDP (User-Defined Problem) class for the Talent Scheduling problem of the 
    from the course AuD(Team02[LG,DSc,RK,DSw])

    Summary
    ========
    The talent scheduling problem can be described as follows. A film producer needs 
    to schedule the scenes of his/her movie on a given location. Each scene has a duration (the days it takes to shoot it) 
    and requires some subset of the cast to be on location. The cast are paid for each day they are required to be on 
    location from the day the first scene they are in is shot, to the day the last scene they are in is shot, even though 
    some of those days they might not be required by the scene currently being shot (i.e., they will be on location waiting
    for the next scene they are in to be shot). Each cast member has a different daily salary. The aim of the film producer
    is to order the scenes in such a way as to minimize the salary cost of the shooting. 

    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.name = ""
        self.num_scenes = 0
        self.num_actors = 0
        self.scenes = []
        self.actor_apprs = []
        self.actor_cost = []
        self.scene_duration = []
        self._load_instance()

    def _load_instance(self):
        """
        Load the instance from the dt-file.
        """
        with open(self.file_path, 'r') as file:
            lines = file.readlines()
            self.name = lines[0].strip()  # First line is the name of the data file
            self.num_scenes = int(lines[1].strip())  # Second line is the number of scenes
            self.num_actors = int(lines[2].strip())  # Third line is the number of actors
            
            # Reading actor data
            actor_data = lines[4:4 + self.num_actors]
            for actor_line in actor_data:
                parts = actor_line.strip().split()
                appearance = list(map(int, parts[:-1]))  # convert each part except last to int
                cost = int(parts[-1])  # Last part is the cost
                self.actor_apprs.append(appearance)
                self.actor_cost.append(cost)

            # Reading scene duration
            self.scene_duration = list(map(int, lines[5 + self.num_actors].strip().split()))



    ####################
    ## Public Methods ##
    ####################

    def info(self):
        print(f"File name: {self.name}")
        print(f"Number of scenes: {self.num_scenes}, Number of actors: {self.num_actors}")
        print(f"Actor appearances: {self.actor_apprs}")
        print(f"Actor costs: {self.actor_cost}")
        print(f"Scene durations: {self.scene_duration}")

    #@njit
    def scene_cost(self):
        # Konvertieren der Liste in ein numpy 2D-Array, für Recheneffizienz
        array_actor_apprs = np.array(self.actor_apprs)
        
        # Array, mit dem multipliziert werden soll
        array_actor_cost = np.array(self.actor_cost)
        
        # Durchführen der Multiplikation mittels Broadcasting für effizienz
        zwischen_ergebnis = array_actor_apprs * array_actor_cost[:, np.newaxis]
        
        array_scene_duration = np.array(self.scene_duration)
        
        # Durchführen der zweiten Multiplikation mittels Broadcasting
        ergebnis = zwischen_ergebnis * array_scene_duration
        
        
        sk=0
        for e in ergebnis:
            sk += e
        #print(f'die Szenen-Kosten {sk}')
        return list(sk)
    
    #@njit
    #######################################Greedy_Primitive##############################################
    def ts_greedy(self):
        sk = self.scene_cost()
        return [i for i in sorted(range(len(sk)), key=lambda x: sk[x], reverse=True)]
    #####################################################################################################

                

    ### this is for the final score ###

    def reorder_lists(self,order):
        actor_apprs = self.actor_apprs
        scene_durations = self.scene_duration
        
        # Neu ordnen der Szenendauern und Auftritte
        new_scene_durations = [scene_durations[i] for i in order]

        new_actor_apprs = [[actor_apprs[j][i] for i in order] for j in range(len(actor_apprs))]
        
        return new_actor_apprs, new_scene_durations


    def extra(self, order):
        actor_apprs, scene_durations = self.reorder_lists(order)
        extra_cost = 0
        for list_index, act_list in enumerate(actor_apprs):
            mod_len = len(act_list)
            while mod_len > 0 and act_list[mod_len - 1] == 0:
                mod_len -= 1

            i = 0
            while i < mod_len - 1:
                if act_list[i] == 1 and act_list[i + 1] == 0:
                    while i + 1 < mod_len and act_list[i + 1] != 1:
                        extra_cost += self.actor_cost[list_index] * scene_durations[i + 1]
                        i += 1
                i += 1
        
        return extra_cost
    


def score(extra):
    sl = film.scene_cost()
    rawcost = sum(sl)
    mincost = rawcost + extra 
    return mincost #+ unnötige Kosten(extra)
    

In [3]:
film = ts_udp(file_path = "/mnt/c/Users/Rehma/Desktop/A&D_Project/data_prj/film116.dat")

film.info()

print(100 * "~")

print(f'die Szenen-Kosten;    {film.scene_cost()}')
print(f'die Greedy-Lösung;    {film.ts_greedy()}')

print("die 'optimale' Lösung;[18, 17, 14, 3, 0, 16, 1, 12, 13, 6, 4, 10, 2, 9, 15, 5, 7, 11, 8]")

File name: film116.dat
Number of scenes: 19, Number of actors: 8
Actor appearances: [[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
Actor costs: [10, 4, 5, 5, 5, 40, 4, 20]
Scene durations: [1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
die Szenen-Kosten;    [40, 19, 49, 30, 9, 45, 5, 10, 10, 53, 15, 9, 18, 9, 20, 40, 20, 25, 5]
die Greedy-Lösung;    [9, 2, 5, 0, 15, 3, 17, 14, 16, 1, 12, 10, 7, 8, 4, 11, 13, 6, 18]
die 'optimale' Lösung;[18, 17, 14, 3, 0, 16

In [4]:
print(f'rowcost; {sum(film.scene_cost())}')
print(f'extra; {film.extra(order = film.ts_greedy())}')
print(f'mincost; {score(film.extra(film.ts_greedy()))} /541')

rowcost; 431
extra; 354
mincost; 785 /541


``` 
Multi-Start-Greedy

- die 1. Szene wird beliebig gewählt
- die Extra-Kosten der restlichen Szenen werden berechnet, SOLANGE der Counter nicht auf Null ist UND
   der Counter aktiviert wurde
- zum Schluss werden die Scenen-Anordnungen mit score() evaluiert
 
```

In [27]:
def zero_plus(start_scene=0):
    extra = 0
    extra_ind_list = []
    for i, appr in enumerate(film.actor_apprs):
        ind = []
        zu_drehen = sum(appr)
        activated = False
        print(f"Schauspieler {i+1}: {appr}")
        print(f"Szenen zu drehen: {zu_drehen}")
        
        for j, a in enumerate(appr):
            if a == 0 and not activated:
                continue
            elif a == 1 and zu_drehen != 0:
                activated = True
                zu_drehen -= 1
            elif a == 0 and zu_drehen != 0 and activated:
                extra += 1
                ind.append(j)
            elif zu_drehen == 0:
                break
        extra_ind_list.append(ind)
        print(f"Extra Szenen: {extra}\n")
    return extra_ind_list

zero_plus(0)

Schauspieler 1: [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
Szenen zu drehen: 5
Extra Szenen: 12

Schauspieler 2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
Szenen zu drehen: 3
Extra Szenen: 14

Schauspieler 3: [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Szenen zu drehen: 8
Extra Szenen: 23

Schauspieler 4: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Szenen zu drehen: 5
Extra Szenen: 23

Schauspieler 5: [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1]
Szenen zu drehen: 7
Extra Szenen: 35

Schauspieler 6: [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Szenen zu drehen: 4
Extra Szenen: 45

Schauspieler 7: [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
Szenen zu drehen: 6
Extra Szenen: 51

Schauspieler 8: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
Szenen zu drehen: 4
Extra Szenen: 65



[[2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15],
 [10, 11],
 [3, 5, 7, 8, 9, 11, 12, 14, 15],
 [],
 [1, 2, 3, 4, 6, 8, 9, 11, 12, 13, 14, 15],
 [3, 4, 6, 7, 8, 10, 11, 12, 13, 14],
 [3, 5, 6, 7, 8, 10],
 [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]]

In [35]:
def create_navi_list():
        extra_prep = []
        for extra_list in extra_ind_list:
            prep_values = [film.scene_duration[idx] for idx in extra_list]  # idx - 1, to convert from 1-based to 0-based
            extra_prep.append(prep_values)
        
        
        # Wir gehen jede Liste in extra_ind_list durch und multiplizieren die Elemente mit den entsprechenden actor_cost Werten
        extra_costs = []
        for i, extra_list in enumerate(extra_prep):
            multiplied_values = [film.actor_cost[i] * value for value in extra_list]
            extra_costs.append(multiplied_values)
        
        extra_costs
        
        # Creating a list of dictionaries
        navi_list = []
        for i in range(len(extra_costs)):
            navi_list.append(dict(zip(extra_ind_list[i], extra_costs[i])))
        
        return navi_list

create_navi_list()

[{2: 10,
  4: 10,
  5: 10,
  6: 10,
  7: 10,
  8: 20,
  9: 10,
  10: 10,
  11: 10,
  13: 10,
  14: 10,
  15: 10},
 {10: 4, 11: 4},
 {3: 5, 5: 5, 7: 5, 8: 10, 9: 5, 11: 5, 12: 5, 14: 5, 15: 5},
 {},
 {1: 5,
  2: 5,
  3: 5,
  4: 5,
  6: 5,
  8: 10,
  9: 5,
  11: 5,
  12: 5,
  13: 5,
  14: 5,
  15: 5},
 {3: 40, 4: 40, 6: 40, 7: 40, 8: 80, 10: 40, 11: 40, 12: 40, 13: 40, 14: 40},
 {3: 4, 5: 4, 6: 4, 7: 4, 8: 8, 10: 4},
 {1: 20,
  2: 20,
  4: 20,
  5: 20,
  6: 20,
  7: 20,
  8: 40,
  9: 20,
  10: 20,
  11: 20,
  12: 20,
  13: 20,
  15: 20,
  16: 20}]

### Instanzen 'small & small02'

```
========================================== film116
.
.
.

mincost = 541, rawcost = 431, extra = 110, bound = 134546788
ccc = 524287,loop = 4980736
reuse = 4456431, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
 18  17  14   3   0  16   1  12  13   6   4  10   2   9  15   5   7  11   8 
----------------------------------------------------------------------------
  .   .   .   X   X   X   X   X   .   .   .   .   .   .   .   .   .   .   . |  10
  .   .   .   .   .   .   .   X   X   -   -   -   -   X   .   .   .   .   . |  4
  .   .   .   .   X   X   X   -   X   X   X   X   X   .   .   .   .   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   X   -   X   -   -   X   X   X |  5
  X   X   -   -   X   X   -   -   -   -   -   X   -   -   -   X   X   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   .   X   X   X   X   .   .   . |  40
  .   .   .   .   .   .   X   X   -   -   X   -   X   X   -   -   -   X   . |  4
  .   X   X   X   X   .   .   .   .   .   .   .   .   .   .   .   .   .   . |  20
----------------------------------------------------------------------------
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   2 
----------------------------------------------------------------------------
  5  25  25  35  40  20  24  28  18  18  18  23  63  58  54  54  14   9  10  = 541
----------------------------------------------------------------------------
  5  25  20  30  40  20  19  18   9   5   9  15  49  53  40  45  10   9  10 
TIME = 776 ms
FINALDATA 19 8 19 541 524287 4456431 0 776 0 tv film116.dat